In [20]:
import spacy
from rank_bm25 import BM25Okapi
import en_core_web_sm
import pandas as pd
import time
import numpy as np
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import sigmoid_kernel

In [8]:
nlp = spacy.load(r"C:\Users\sachinika.pathirana_\Desktop\Personal\Projects\search_engine\Mamaearth-Product-Analysis\en_core_web_sm\en_core_web_sm-3.0.0")

C:\Users\sachinika.pathirana_\anaconda3\lib\site-packages\spacy\util.py:887: UserWarning: [W095] Model 'en_core_web_sm' (3.0.0) was trained with spaCy v3.0 and may not be 100% compatible with the current version (3.5.2). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [9]:
df = pd.read_csv("Mamaearth.csv")
df = df.drop_duplicates()

# Product Search (Sementic)

In [10]:
text_list = df.Product_Name.str.lower().values
tok_text=[]

In [11]:
for doc in nlp.pipe(text_list, disable=["tagger", "parser","ner"]):
    tok = [t.text for t in doc if t.is_alpha]
    tok_text.append(tok)

C:\Users\sachinika.pathirana_\anaconda3\lib\site-packages\spacy\pipeline\lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


In [12]:
bm25 = BM25Okapi(tok_text)

In [16]:
query= input("user:")
tokenized_query = query.lower().split(" ")
results = bm25.get_top_n(tokenized_query, df.Product_Name.values, n=1)
for i in results:
    print(i)

user: hair loss


Onion Hair Oil for Hair Regrowth & Hair Fall Control


# Product Recommendation

In [1]:
def get_type():
    '''
    answers: hair, skin, body
    '''
    type = input("Where do you have the problem?")
    type = " ".join(["".join(n.split()) for n in type.lower().split(',')])
    return type

def get_major():
    '''
    answers: acne, glowing, hair fall
    '''
    major = input("what is the major concern?")
    major = " ".join(["".join(n.split()) for n in major.lower().split(',')])
    return major

def get_product():
    '''
    answers : face wash, shampoo, serum
    '''
    product = input("What kind of product do you want?")
    product = " ".join(["".join(n.split()) for n in product.lower().split(',')])
    return product

def get_searchTerms():
  searchTerms = [] 

  type = get_type()
  if type != 'skip':
    searchTerms.append(type)

  major = get_major()
  if major != 'skip':
    searchTerms.append(major)

  product = get_product()
  if product != 'skip':
    searchTerms.append(product)
    
  return searchTerms

In [18]:
data = df[["Product_Name","Tokens"]]
data.head()

,Product_Name,Tokens
0,Tea Tree Facewash for acne & pimples,tea tree facewash acne pimple
1,Anti Hair Fall Kit for complete hair care,anti hair fall kit complete hair care
2,Argan Hair Mask with Argan Avocado Oil and Mil...,argan hair mask argan avocado oil milk protein...
3,Ubtan Bamboo Sheet Mask with Turmeric & Saffro...,ubtan bamboo sheet mask turmeric saffron skin ...
4,Niacinamide Bamboo Sheet Mask with Niacinamide...,niacinamide bamboo sheet mask niacinamide ging...


## Cosine Similarity

In [27]:
def make_recommendation(data=data):
  new_row = data.iloc[-1,:].copy() 
  
  searchTerms = get_searchTerms()  
  new_row.iloc[-1] = " ".join(searchTerms) 
  
  data = data.append(new_row)
  
  count = CountVectorizer(stop_words='english')
  count_matrix = count.fit_transform(data['Tokens'])

  cosine_sim2 = cosine_similarity(count_matrix, count_matrix) 
  #print(cosine_sim2)
  
  sim_scores = list(enumerate(cosine_sim2[-1,:]))
  sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
  #print(sim_scores)

  ranked_titles = []
  for i in range(1, 4):
    indx = sim_scores[i][0]
    ranked_titles.append(data['Product_Name'].iloc[indx])
  
  return ranked_titles


make_recommendation()

Where do you have the problem? skin
what is the major concern? fairness
What kind of product do you want? cream


C:\Users\sachinika.pathirana_\AppData\Local\Temp\ipykernel_36012\3027457101.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(new_row)


['AntiPollution Daily Face Cream for Dry and Oily Skin with Turmeric and PollustopÂ® For a Bright Glowing Skin ',
 'Vitamin C Face Cream with Vitamin C & SPF 20 for Skin Illumination',
 'Skin Illuminate Face Serum for Radiant Skin with Vitamin C & Turmeric â€“']

## Sigmoid Kernal

In [29]:
def make_recommendation2(data=data):
  new_row = data.iloc[-1,:].copy() 
  
  searchTerms = get_searchTerms()  
  new_row.iloc[-1] = " ".join(searchTerms) 
  
  data = data.append(new_row)
  
  count = CountVectorizer(stop_words='english')
  count_matrix = count.fit_transform(data['Tokens'])

  sig_sim2 = sigmoid_kernel(count_matrix, count_matrix) 
  #print(sig_sim2)
  
  sim_scores = list(enumerate(sig_sim2[-1,:]))
  sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
  #print(sim_scores)

  ranked_titles = []
  for i in range(1, 4):
    indx = sim_scores[i][0]
    ranked_titles.append(data['Product_Name'].iloc[indx])
  
  return ranked_titles


make_recommendation2()

Where do you have the problem? skin
what is the major concern? fairness
What kind of product do you want? cream


C:\Users\sachinika.pathirana_\AppData\Local\Temp\ipykernel_36012\3851944904.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(new_row)


['Hydrating Natural Body Lotion with Cucumber & Aloe Vera for Normal Skin ',
 'Spotless Skin Combo: Ubtan Facewash  100ml and Bye Bye Blemishes Face Cream ',
 'Vitamin C Face Cream with Vitamin C & SPF 20 for Skin Illumination']